In [ ]:
! pip install openai langchain unstructured chromadb Cython tiktoken pypdf

In [45]:
#Loading all the library and packages

import os
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


os.environ["OPENAI_API_KEY"] = 'your_openai_api_key'

davinci = OpenAI(model_name='text-davinci-003')

In [51]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
file1= '/content/drive/MyDrive/sample-statement-of-work.pdf'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
#Read the PDF File
loader = PyPDFLoader(file1)
sample_doc = loader.load()

In [53]:
# Get OpenAI embeddings and create the vector database
#Use OpenAI embeddings
embeddings = OpenAIEmbeddings()

# create a vector database using the sample document
vectordb = Chroma.from_documents(
    documents=sample_doc, embedding=embeddings, persist_directory="chromadb"
)

vectordb.persist()

In [ ]:
# create the conversation and loop

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
pdf_chat = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.8) , vectordb.as_retriever(), memory=memory)
while True:
    line = input("Enter a question: ")
    if line == "":
        break
    result = pdf_chat({"question": line})
    print(f"Answer: {result['answer']}")

Enter a question: what is fees?
Answer:  Fees are payments for goods or services. In this context, fees refer to payments for services provided by Globex, such as consulting services, and payments for specific customer transactions like international money transfers.
Enter a question: what are the rates?
Answer:  The fees/rates for services provided by Globex are €1,200/day for developers, consultants, test managers, education trainers, and project managers. The fees/rates for customer transactions such as international money transfers are not specified in the context.
Enter a question: can you calculate the average fees?
Answer:  The average fee for services provided by Globex is €1,200 per day. The fee for customer transactions such as international money transfers is not specified in this document.
Enter a question: list out all the risk
Answer:  The identified risks associated with this project are adoption of the software, deployment of changes, and resource availability.
